In [1]:
from xcube.core.store import new_data_store
import os

In [3]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(
    n_workers=4,
    threads_per_worker=2,
    memory_limit="4GB",   # per worker
)
client = Client(cluster)

In [4]:
credentials = {
"key":"",
"secret": ""
}

In [5]:
store = new_data_store("stac-cdse-ardc", **credentials)

In [13]:
from xcube_stac.utils import reproject_bbox

bbox=[-6.3, 53.2, -3.2, 55.3]
bbox=[-3.3,55.9,-2.3, 56.2] # Edinburgh
bbox=[4.7,58.6,6.7, 60.6] # Stavanger
bbox=[4.7,58.6,6.1, 59.6] # Stavanger
bbox_wgs84 = [7.7, 55.5, 8.4, 56.1] #Esbjerg
bbox_wgs84 = [4.3, 59.0, 5.7, 59.6] #north of Stavanger


crs_utm = "EPSG:32632"
bbox_utm = reproject_bbox(bbox_wgs84, "EPSG:4326", crs_utm)
bbox_utm

(230116.44269369252, 6544732.814255137, 313766.4732013203, 6616254.075772593)

In [14]:
store.list_data_ids()

['sentinel-2-l2a', 'sentinel-2-l1c', 'sentinel-3-syn-2-syn-ntc']

In [15]:
%%time
ds = store.open_data(
    "sentinel-2-l2a",
    #time_range=("2022-08-07", "2022-08-16"),
    time_range=("2023-06-07", "2023-06-16"), # stavanger example 
    bbox=bbox_utm,
    spatial_res=10,
    crs=crs_utm,
    asset_names=["B02", "B03", "B04", "B08", "SCL"]
)
ds

CPU times: user 2min 7s, sys: 2.41 s, total: 2min 10s
Wall time: 2min 33s


<xarray.Dataset> Size: 7GB
Dimensions:      (time: 6, x: 8366, y: 7153)
Coordinates:
  * time         (time) datetime64[ns] 48B 2023-06-07T10:56:21.024000 ... 202...
  * x            (x) float64 67kB 2.301e+05 2.301e+05 ... 3.138e+05 3.138e+05
  * y            (y) float64 57kB 6.616e+06 6.616e+06 ... 6.545e+06 6.545e+06
    spatial_ref  int64 8B 0
Data variables:
    B02          (time, y, x) float32 1GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    B03          (time, y, x) float32 1GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    B04          (time, y, x) float32 1GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    B08          (time, y, x) float32 1GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    SCL          (time, y, x) float32 1GB dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
Attributes:
    stac_item_ids:       {'2023-06-07T10:56:21.024000': ['S2A_MSIL2A_20230607...
    stac_catalog_url:    https://stac.dataspace.copernicus.eu/v1
    xcube_stac_version:  1.1.1

In [3]:
S3_USER_STORAGE_KEY = os.environ["S3_USER_STORAGE_KEY"]
S3_USER_STORAGE_SECRET = os.environ["S3_USER_STORAGE_SECRET"]
S3_USER_STORAGE_BUCKET = os.environ["S3_USER_STORAGE_BUCKET"]

In [4]:
from xcube.core.store import new_data_store
team_store = new_data_store("s3", 
                       root=S3_USER_STORAGE_BUCKET, 
                       storage_options=dict(anon=False, 
                                            key=S3_USER_STORAGE_KEY, 
                                            secret=S3_USER_STORAGE_SECRET))

list(team_store.get_data_ids())

['cmems_4km_2022080317.zarr',
 'cmems_4km_2023060717.zarr',
 'cmems_7km_2022080317.zarr',
 'cmems_7km_2023060717.zarr',
 'sen2l2a_edinburg_2022080712.zarr',
 'sen2l2a_esbjerg_2022080716.zarr',
 'sen2l2a_stavanger_2022080712_2.zarr',
 'sen2l2a_stavanger_2023060716.zarr',
 'sen2l2a_stavanger_2023060716_2.zarr']

In [17]:
%%time
team_store.write_data(ds,'sen2l2a_stavanger_2023060716_2.zarr', replace=True)

/home/conda/users/30f6ffd2-1762860680-159-deepesdl-xcube-1.12.0/lib/python3.13/site-packages/xcube/core/store/fs/impl/dataset.py:200: UserWarning: Times can't be serialized faithfully to int64 with requested units 'seconds since 1970-01-01'. Resolution of 'milliseconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  data.to_zarr(
/home/conda/users/30f6ffd2-1762860680-159-deepesdl-xcube-1.12.0/lib/python3.13/site-packages/dask/array/chunk.py:139: RuntimeWarning: Mean of empty slice
  return reduction(x.reshape(newshape), axis=tuple(range(1, x.ndim * 2, 2)), **kwargs)


CPU times: user 24min 45s, sys: 45.4 s, total: 25min 30s
Wall time: 47min 22s


'sen2l2a_stavanger_2023060716_2.zarr'